<img src="https://images.efollett.com/htmlroot/images/templates/storeLogos/CA/864.gif" style="float: right;"> 




# ECON611
### HW_4 -  SQL
#### Due: Nov 18

We will be using the sample databases that come from the DBeaver program (use the Database Installation guide to assist you on installing it). We will be using the sample tables that are provided there. Save your work as a SQL script with answers to specific questions commented out properly labled.

## 1. Music and Album Data
---
We will be using the following tables from DBeaver:
• Album • Artist • Genre • Track • PlaylistTrack • MediaType

## Problems (40 points)
---

#### A. (5 points). Output a table that has the columns Album Title, Artist, Track Name, Genre, Composer, Length of Song (minutes) 
---

In [2]:
SELECT A.Title AS Album_Title, T.Name AS Track_Name
	  ,(T.Milliseconds/1000) AS Song_Minutes
	  , Art.Name AS Artist_Name, G.Name AS Genre_Name
FROM Artist AS Art
INNER JOIN Album AS A
ON Art.ArtistId = A.ArtistId
INNER JOIN Track AS T 
ON T.AlbumId = A.AlbumId
INNER JOIN Genre AS G 
ON G.GenreId = T.GenreId
ORDER BY Artist_Name;

SyntaxError: invalid syntax (<ipython-input-2-f9f59ac29ab0>, line 1)

#### B. (5 points). Output a frequency table for how many letters are in an album title and a song title. (i.e. 3 columns: number of letters, number of album titles with that amount of letters, number of song titles with that amount of letters). What is the average and range of amount of letters?
---

In [ ]:
SELECT 
	finaltable.songtitle, finaltable.Title, finaltable.numberofletters, finaltable.numberoflettersong, finaltable.freqalbumlength, 
	Count(finaltable.songtitle) OVER (PARTITION BY finaltable.numberoflettersong ORDER BY finaltable.numberoflettersong) as freqsonglength -- Number of songs with same title length
FROM(
	SELECT 
		Track.AlbumId, freqalbum.Title, 
		freqalbum.numberofletters, freqalbum.freqalbumlength, 
		Track.Name as songtitle, length(Track.Name) as numberoflettersong
	FROM(
		SELECT *, Count(*) OVER (PARTITION BY numberofletters ORDER BY numberofletters) as freqalbumlength
		FROM(
			SELECT *, length(Title) as numberofletters
			From Album
			) albumlength -- Length of Album Title
		ORDER BY numberofletters) freqalbum -- Number of albums with same title length
	JOIN Track
	on Track.AlbumId = freqalbum.AlbumId) finaltable -- Length of Song Title and join with Album
ORDER BY 3, 4, 5, 6 DESC; -- Order by lengths and frequencies

SELECT 
	AVG(numberofletters) as avg_albumtitle, AVG(numberoflettersong) as avg_songtitle, 
	MAX(numberofletters)-MIN(numberofletters) as range_albumtitle, MAX(numberoflettersong)-MIN(numberoflettersong) as range_albumtitle
FROM(
	SELECT 
		finaltable.songtitle, finaltable.Title, finaltable.numberofletters, finaltable.numberoflettersong, finaltable.freqalbumlength, 
		Count(finaltable.songtitle) OVER (PARTITION BY finaltable.numberoflettersong ORDER BY finaltable.numberoflettersong) as freqsonglength -- Number of songs with same title length
	FROM(
		SELECT 
			Track.AlbumId, freqalbum.Title, 
			freqalbum.numberofletters, freqalbum.freqalbumlength, 
			Track.Name as songtitle, length(Track.Name) as numberoflettersong
		FROM(
			SELECT *, Count(*) OVER (PARTITION BY numberofletters ORDER BY numberofletters) as freqalbumlength
			FROM(
				SELECT *, length(Title) as numberofletters
				From Album
				) albumlength -- Length of Album Title
			ORDER BY numberofletters) freqalbum -- Number of albums with same title length
		JOIN Track
		on Track.AlbumId = freqalbum.AlbumId) finaltable -- Length of Song Title and join with Album
	ORDER BY 3, 4, 5, 6 DESC);

#### C. (10 points). How many songs did each composer write? How many artists composed their own music? Output all of the Artists who had songs that were longer than 2.5 minutes and they did not write their own songs.
---

In [ ]:
SELECT COUNT(*), Composer
FROM Track
WHERE Composer NOTNULL
GROUP BY Composer
ORDER BY COUNT(*) DESC;
​
SELECT COUNT(*)
FROM(
	SELECT lastjoin.*, Genre.Name as genre
	FROM (
		SELECT albumartist.*, Track.Name as SongTitle, Track.GenreId, Track.Composer, Track.Milliseconds/1000/60 as song_length
		FROM(
			SELECT Album.AlbumId, Album.ArtistId, 
			Album.Title as AlbumTitle, 
			Artist.Name as ArtistName
			FROM Album
			JOIN Artist
			on Album.ArtistId = Artist.ArtistId
			group by Artist.ArtistId) albumartist --Inner join Album and Artist
		JOIN Track
		on albumartist.AlbumId = Track.AlbumId
		group by albumartist.AlbumId) lastjoin -- Inner join the previous join and Track
	JOIN Genre
	on lastjoin.GenreId = Genre.GenreId -- Lastly join the previous join with Genre
	group by lastjoin.ArtistId
	)
WHERE ArtistName = Composer; -- Where Artists and Composer are the same

SELECT *
FROM(
	SELECT lastjoin.*, Genre.Name as genre
	FROM (
		SELECT albumartist.*, Track.Name as SongTitle, Track.GenreId, Track.Composer, Track.Milliseconds/1000/60 as song_length
		FROM(
			SELECT Album.AlbumId, Album.ArtistId, 
			Album.Title as AlbumTitle, 
			Artist.Name as ArtistName
			FROM Album
			JOIN Artist
			on Album.ArtistId = Artist.ArtistId
			group by Artist.ArtistId) albumartist --Inner join Album and Artist
		JOIN Track
		on albumartist.AlbumId = Track.AlbumId
		group by albumartist.AlbumId) lastjoin -- Inner join the previous join and Track
	JOIN Genre
	on lastjoin.GenreId = Genre.GenreId -- Lastly join the previous join with Genre
	group by lastjoin.ArtistId
	)
WHERE ArtistName != Composer AND song_length > 2.5
ORDER BY song_length desc;

#### D. (10 points). How many albums contain songs that only have an even number of seconds in the length of the song and the songs are all MPEG audio files?
---

In [ ]:
SELECT COUNT(*)
FROM( 
	SELECT lastjoin.*, Genre.Name as genre
	FROM (
		SELECT albumartist.*, Track.Name as SongTitle, Track.GenreId, Track.Composer, Track.Milliseconds/1000 as song_length_seconds, Track.MediaTypeId -- added media type, changed to length in seconds
		FROM(
			SELECT Album.AlbumId, Album.ArtistId, 
			Album.Title as AlbumTitle, 
			Artist.Name as ArtistName
			FROM Album
			JOIN Artist
			on Album.ArtistId = Artist.ArtistId
			group by Artist.ArtistId) albumartist --Inner join Album and Artist
		JOIN Track
		on albumartist.AlbumId = Track.AlbumId
		group by albumartist.AlbumId) lastjoin -- Inner join the previous join and Track
	JOIN Genre
	on lastjoin.GenreId = Genre.GenreId -- Lastly join the previous join with Genre
	group by lastjoin.ArtistId
	)
WHERE (song_length_seconds%2 = 0) AND (MediaTypeId = 1 or MediaTypeId = 3) -- Even number of seconds AND MPEG file types
ORDER BY song_length_seconds;

#### E. (10 points). How large, in terms of Megabytes are each album? On average, how large, in terms of bytes, is a pop song? What is the range and average duration of a heavy metal song in minutes?
---

In [ ]:
SELECT *, Bytes/1000000 as Megabytes -- Divide Bytes by 1mil to get Megabytes
FROM(
	SELECT lastjoin.*, Genre.Name as genre
	FROM (
		SELECT albumartist.*, Track.Name as SongTitle, Track.GenreId, Track.Composer, Track.Milliseconds/1000 as song_length_seconds, Track.MediaTypeId, Track.Bytes -- added bytes
		FROM(
			SELECT Album.AlbumId, Album.ArtistId, 
			Album.Title as AlbumTitle, 
			Artist.Name as ArtistName
			FROM Album
			JOIN Artist
			on Album.ArtistId = Artist.ArtistId
			group by Artist.ArtistId) albumartist --Inner join Album and Artist
		JOIN Track
		on albumartist.AlbumId = Track.AlbumId
		group by albumartist.AlbumId) lastjoin -- Inner join the previous join and Track
	JOIN Genre
	on lastjoin.GenreId = Genre.GenreId -- Lastly join the previous join with Genre
	group by lastjoin.ArtistId
	);

SELECT AVG(Bytes) -- Average number of bytes
FROM(
	SELECT lastjoin.*, Genre.Name as genre
	FROM (
		SELECT albumartist.*, Track.Name as SongTitle, Track.GenreId, Track.Composer, Track.Milliseconds/1000 as song_length_seconds, Track.MediaTypeId, Track.Bytes
		FROM(
			SELECT Album.AlbumId, Album.ArtistId, 
			Album.Title as AlbumTitle, 
			Artist.Name as ArtistName
			FROM Album
			JOIN Artist
			on Album.ArtistId = Artist.ArtistId
			group by Artist.ArtistId) albumartist --Inner join Album and Artist
		JOIN Track
		on albumartist.AlbumId = Track.AlbumId
		group by albumartist.AlbumId) lastjoin -- Inner join the previous join and Track
	JOIN Genre
	on lastjoin.GenreId = Genre.GenreId -- Lastly join the previous join with Genre
	group by lastjoin.ArtistId
	)
WHERE genre = 'Pop'; -- Only for pop songs

SELECT AVG(song_length), MAX(song_length)-MIN(song_length) as song_length_range
FROM(
	SELECT lastjoin.*, Genre.Name as genre
	FROM (
		SELECT albumartist.*, Track.Name as SongTitle, Track.GenreId, Track.Composer, Track.Milliseconds/1000/60 as song_length, Track.MediaTypeId, Track.Bytes -- back to minutes
		FROM(
			SELECT Album.AlbumId, Album.ArtistId, 
			Album.Title as AlbumTitle, 
			Artist.Name as ArtistName
			FROM Album
			JOIN Artist
			on Album.ArtistId = Artist.ArtistId
			group by Artist.ArtistId) albumartist --Inner join Album and Artist
		JOIN Track
		on albumartist.AlbumId = Track.AlbumId
		group by albumartist.AlbumId) lastjoin -- Inner join the previous join and Track
	JOIN Genre
	on lastjoin.GenreId = Genre.GenreId -- Lastly join the previous join with Genre
	group by lastjoin.ArtistId
	)
WHERE genre = 'Metal'; -- Only for metal music

## 2. Customer Data
---
We will be using the following tables from DBeaver:
• Customer • Invoice • InvoiceLine • Album • Artist • Genre • Track • PlaylistTrack • MediaType

## Problems (40 points)
---

#### A. (5 points). Output a table that shows the number of customers in each city, and also have shows the state and country that the city is located in.
---

In [ ]:
SELECT City, State, Country, COUNT(*)
FROM Customer 
GROUP BY City
ORDER BY Count(*) DESC;

#### B. (10 points). How much in total was spent on music per year? What is the average amount a person spent on music in a year? What is the average amount a person with a gmail account spent on music in a year?
---

In [ ]:
SELECT SUM(Total) as totalspent, STRFTIME('%Y', InvoiceDate)
FROM Invoice i 
GROUP BY STRFTIME('%Y', InvoiceDate)
ORDER BY STRFTIME('%Y', InvoiceDate) ASC; -- This gives the total for each year

SELECT totalspent/COUNT(DISTINCT CustomerId) as avgspending
FROM(
	SELECT *, SUM(Total) as totalspent, STRFTIME('%Y', InvoiceDate)
	FROM Invoice i 
	GROUP BY STRFTIME('%Y', InvoiceDate)
	ORDER BY STRFTIME('%Y', InvoiceDate) ASC
	); -- This divides the totalspent by the number of unique customers, thus giving average spending

SELECT totalspent/COUNT(DISTINCT CustomerId) as avgspending
FROM(
	SELECT *, SUM(Total) as totalspent, STRFTIME('%Y', InvoiceDate)
	FROM(
		SELECT c.Email, i.*
		From Customer c
		JOIN Invoice i
		on c.CustomerId=i.CustomerId
		WHERE c.Email LIKE '%@gmail.com'
		)
	GROUP BY CustomerId
	); -- This used the same process, but truncated our sample to only those emails ending in '@gmail.com'

#### C. (10 points). What year-month combination had the most spending on music? Which year-month combi- nation had the most money spent on pop music?
---

In [ ]:
SELECT SUM(Total) as totalspent, STRFTIME('%Y %m', InvoiceDate)
FROM Invoice
GROUP BY STRFTIME('%Y %m', InvoiceDate)
ORDER BY totalspent DESC
LIMIT 1;

-- Found out how to join multiple tables!
SELECT SUM(Total) as totalspent, STRFTIME('%Y %m', InvoiceDate)
FROM(
	SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*
	FROM Invoice
	JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
	JOIN Track ON InvoiceLine.TrackId = Track.TrackId
	JOIN Genre ON Genre.GenreId = Track.GenreId
	WHERE Genre.Name = 'Pop') poponly
GROUP BY STRFTIME('%Y %m', InvoiceDate)
ORDER BY totalspent DESC
LIMIT 1;

#### D. (10 points). What genre has the most songs bought that are in the pop genre? What is the total money spent on these songs? Which customer is spending the most on pop songs?
---

In [ ]:
SELECT SUM(p.Quantity), Artist.Name as artist -- Displaying number of purchased songs and artist name
FROM(
	SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*
	FROM Invoice
	JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
	JOIN Track ON InvoiceLine.TrackId = Track.TrackId
	JOIN Genre ON Genre.GenreId = Track.GenreId
	WHERE Genre.Name = 'Pop'
	) p
JOIN Album ON p.AlbumId = Album.AlbumId
JOIN Artist ON Album.ArtistId = Artist.ArtistId -- Added Album and Artist to our megatable
GROUP BY artist
ORDER BY SUM(p.Quantity) DESC
LIMIT 1; -- We only want this artist with the greatest number of purchases (ie. U2)

SELECT SUM(DISTINCT Total) -- Gives total spent on Pop songs
FROM(
	SELECT *
	FROM(
		SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*
		FROM Invoice
		LEFT JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
		JOIN Track ON InvoiceLine.TrackId = Track.TrackId
		JOIN Genre ON Genre.GenreId = Track.GenreId
		WHERE Genre.Name = 'Pop'
		) p
	JOIN Album ON p.AlbumId = Album.AlbumId
	JOIN Artist ON Album.ArtistId = Artist.ArtistId
	) purchasedpop;

SELECT SUM(DISTINCT Total), CustomerId
FROM(
	SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*
	FROM Invoice
	JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
	JOIN Track ON InvoiceLine.TrackId = Track.TrackId
	JOIN Genre ON Genre.GenreId = Track.GenreId
	WHERE Genre.Name = 'Pop'
		)
GROUP BY CustomerId
ORDER BY SUM(DISTINCT Total) DESC
LIMIT 4;

#### E. (10 points). Output how many songs each artist is selling in each country? Who is selling the post outside of the US? On average, how many songs is each artist selling outside of the US?
---

In [ ]:
SELECT ArtistId, artist, BillingCountry, SUM(Quantity) as quantity 
FROM(
	SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*, Album.*, Artist.*, Artist.Name as artist
	FROM Invoice
	JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
	JOIN Track ON InvoiceLine.TrackId = Track.TrackId
	JOIN Genre ON Genre.GenreId = Track.GenreId
	JOIN Album ON Track.AlbumId = Album.AlbumId
	JOIN Artist ON Album.ArtistId = Artist.ArtistId
	) wowza
GROUP BY BillingCountry, ArtistId
ORDER BY ArtistId ASC, SUM(Quantity) DESC, BillingCountry ASC; -- Gives how many songs each artist sells in each country

SELECT ArtistId, artist, SUM(quantity) -- Gives total sold
FROM(
	SELECT ArtistId, artist, BillingCountry, SUM(Quantity) as quantity
	FROM(
		SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*, Album.*, Artist.*, Artist.Name as artist
		FROM Invoice
		JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
		JOIN Track ON InvoiceLine.TrackId = Track.TrackId
		JOIN Genre ON Genre.GenreId = Track.GenreId
		JOIN Album ON Track.AlbumId = Album.AlbumId
		JOIN Artist ON Album.ArtistId = Artist.ArtistId
		) wowza
	GROUP BY BillingCountry, ArtistId
	ORDER BY ArtistId ASC, SUM(Quantity) DESC, BillingCountry ASC
	) woooowza
WHERE BillingCountry is not 'USA' -- Limits to outside of USA
GROUP BY ArtistId
ORDER BY quantity DESC;

SELECT AVG(totalnotus) -- Gives average sold not in the USA
FROM (
	SELECT ArtistId, artist, SUM(quantity) as totalnotus
	FROM(
		SELECT ArtistId, artist, BillingCountry, SUM(Quantity) as quantity
		FROM(
			SELECT Invoice.*, InvoiceLine.*, Track.*, Genre.*, Album.*, Artist.*, Artist.Name as artist
			FROM Invoice
			JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
			JOIN Track ON InvoiceLine.TrackId = Track.TrackId
			JOIN Genre ON Genre.GenreId = Track.GenreId
			JOIN Album ON Track.AlbumId = Album.AlbumId
			JOIN Artist ON Album.ArtistId = Artist.ArtistId
			) wowza
		GROUP BY BillingCountry, ArtistId
		ORDER BY ArtistId ASC, SUM(Quantity) DESC, BillingCountry ASC
		) woooowza
	WHERE BillingCountry is not 'USA'
	GROUP BY ArtistId
	ORDER BY quantity DESC
	) wooooooooowza;

## 3. Window Functions (20 points)
---

#### A. (10 points). Return a list of Customers that assigns a sequential integer to each customer and resets the number when the country of the customer changes for Customers are not from Canada and Brazil and whose email address have `yahoo` or `gmail'
---

In [ ]:
SELECT *, ROW_NUMBER() OVER (PARTITION BY Country)
FROM Customer c
WHERE Country is not 'Canada' OR Country is not 'Brazil' 
	AND (Email LIKE '%yahoo.com' OR Email LIKE '%gmail.com')
ORDER BY Country;

#### B. (10 points). Compute the rank for each track in each album based on the track’s length, then append all the tracks that have a lengthrank of 2 and 4 
---

In [ ]:
SELECT *
FROM(
	SELECT AlbumID, Name, Milliseconds, 
		DENSE_RANK() OVER (PARTITION BY AlbumId ORDER BY Milliseconds) lengthrank --This is the length ranking function
	FROM Track)
WHERE lengthrank = 2 OR lengthrank = 4;

## BONUS (20 points)
---
- Using SQLITE syntax [make your own SQLITE table](https://www.sqlitetutorial.net/sqlite-create-table/) and set of data. You will need to create a table of 5 columns and 10 rows containing any information you find interesting. (5 points)
- You should create/import your table need to [dbeaver](https://docs.roguewave.com/en/zendstudio/current/content/create_manage_database_with_dbeaver.htm). (5 points)
- Perform 3 different calculations using your data in order to tell a story of your data set. (10 points)

** FEEL FREE TO USE/EXPLORE OTHER SOURCES**